In [3]:
import time, os
from datetime import datetime, timedelta
from utils.generate_data import generate_nyctaxistream_async,generate_locationid_temperature_async
from utils.kafka import write_to_kafka_async,list_kafka_topics,delete_kafka_topic,create_kafka_topic,close_kafka_admin,clean_reset_kafka_topics
from utils.spark import read_kafka,read_spark_parquet,write_spark_stream
import asyncio
from delta.tables import DeltaTable
from pyspark.sql.functions import to_timestamp
from utils import Startup
from types import SimpleNamespace


In [4]:
startup = Startup()
session_vars = SimpleNamespace(**startup.__dict__)
spark = session_vars.spark

# spark = session.spark


Resetting kafka topics...
Deleting topic nyctaxistream
Topic 'nyctaxistream' deleted
Starting Spark App: spark_session_6259ef2a


26/01/02 12:00:49 WARN Utils: Your hostname, Souravs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
26/01/02 12:00:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/souravagasti/.ivy2/cache
The jars for the packages stored in: /Users/souravagasti/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.spark#spark-token-provider-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8b55bd7-aea2-4e68-a17a-48cdd0afe9d9;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central


:: loading settings :: url = jar:file:/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.5.0 in central
	found com.github.luben#zstd-jni;1.5.5-1 in central
	found org.lz4#lz4-java;1.8.0 in central
:: resolution report :: resolve 172ms :: artifacts dl 6ms
	:: modules in use:
	com.github.luben#zstd-jni;1.5.5-1 from central in [default]
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [defa

Spark session initialized successfully with Delta support.


In [19]:
# Check if Delta table exists
# If not, read raw parquet data and write as Delta table
# put this in a util (one-time activity)
try:
    if not(DeltaTable.isDeltaTable(spark, session_vars.delta_path)):
        required_cols = [
            "hvfhs_license_num",
            "dispatching_base_num",
            "originating_base_num",
            "request_datetime",
            "on_scene_datetime",
            "pickup_datetime",
            "dropoff_datetime",
            "PULocationID",
            "DOLocationID",
            "trip_miles",
            "trip_time"
        ]

        rnyctaxi_cols = [
            "hvfhs_license_num",
            "dispatching_base_num",
            "pickup_datetime",
            "dropoff_datetime",
            "PULocationID",
            "DOLocationID"
        ]

        spark.read.parquet(session_vars.raw_file_path).select(*rnyctaxi_cols)

        df_raw = read_spark_parquet(spark, session_vars.raw_file_path).select(*required_cols).write.format("delta").mode("overwrite").save(session_vars.delta_path)
        
        print(f"Delta table initiated at {session_vars.delta_path}")
    else:
        print(f"Delta table already exists at {session_vars.delta_path}")
except Exception as e:
    print(f"Error checking/creating Delta table: {e}")

finally:
    # Register the Delta table as a temp view
    spark.read.format("delta").load(session_vars.delta_path).createOrReplaceTempView("delta_table")

Delta table already exists at /Users/souravagasti/projects/real-time-data/data/nyctaxi/delta


call the event generation notebook here--->

...and then proceed

In [5]:
# spark.sql("delete from delta_table where dispatching_base_num IN ('B03494', 'B03496')")

In [5]:
#startingOffset = Earliest to read from beginning
# read from kafka topic

params = {
    "spark": spark,
    "topic": "nyctaxistream",
    "schema": session_vars.schema,
    "startingOffsets": "earliest"
}

df_stream = read_kafka(**params)

# cast the timestamp cols
df_stream = (
    df_stream
    .withColumn("pickup_datetime", to_timestamp("pickup_datetime"))
    .withColumn("dropoff_datetime", to_timestamp("dropoff_datetime"))
)

Those new records generated in kafka should be caught in this data stream. 
To read from it, write it to memory and see the contents

In [ ]:
# write to memory and select * from "topic" to inspect content or write to console
params = {
"df": df_stream,
"spark": spark,
"topic": "nycstream",       #the in-memory df would be of this name
"write_to": "console",
"trigger_mode": "processingTime", #once, continuous
"output_path": session_vars.delta_path,
"output_mode": "update",
"merge_keys": session_vars.merge_keys,
"interval_seconds": 5,
}

query = write_spark_stream(**params)
# while running:
# print("offset just written: ",query.lastProgress)#["sources"][0]["endOffset"])

# Wait until the query finishes
try:
    query.awaitTermination()
    # spark.sql("select * from nycstream").show(10)
    # if params_write_spark_stream["trigger_mode"] == "continuous": 
    #     spark.sql("select * from nycstream").show(10)
finally:
    if params["trigger_mode"] != "continuous":
        query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------+--------------------+-------------------+-------------------+------------+------------+----------+---------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|
+-----------------+--------------------+-------------------+-------------------+------------+------------+----------+---------+
|          HV00007|              B03494|2025-01-01 00:01:50|2025-01-01 00:02:25|         100|         100|      NULL|     NULL|
|          HV00003|              B03496|2025-01-01 00:15:56|2025-01-01 01:00:12|         120|         150|      NULL|     NULL|
|          HV00005|              B03496|2025-01-01 00:15:56|2025-01-01 01:00:12|         120|         150|      NULL|     NULL|
|          HV00002|              B03494|2025-01-01 00:16:45|2025-01-01 00:19:08|         100|         100|      NULL|     NULL|
|      

inspect values of stream

In [1]:
spark.sql("select count(*) from nycstream").show()

NameError: name 'spark' is not defined

lets write this stream to delta. first lets check if these records exist on the delta table

In [37]:
# spark.sql("delete from delta_table where dispatching_base_num IN ('B03494', 'B03496')")

count = spark.sql("""
    SELECT COUNT(*) AS cnt
    FROM delta_table
    WHERE dispatching_base_num IN ('B03494', 'B03496')
""").collect()[0]["cnt"]

print("Count:", count)

Count: 26


lets merge the stream into delta

In [47]:
params_write_spark_stream = {
"df": df_stream,
"spark": spark,
"topic": "nycstream",
"write_to": "delta",
"trigger_mode": "once",
"output_path": session_vars.delta_path,
"output_mode": "update",
"merge_keys": session_vars.merge_keys,
"interval_seconds": 5,
}

query = write_spark_stream(**params_write_spark_stream)
# print(query.lastProgress)
#merge these in!

try:
    query.awaitTermination()
finally:
    if params_write_spark_stream["trigger_mode"] != "continuous":
        query.stop()

Started merge stream with foreachBatch.


In [48]:

count = spark.sql("""
    SELECT COUNT(*) AS cnt
    FROM delta_table
    WHERE dispatching_base_num IN ('B03494', 'B03496')
""").collect()[0]["cnt"]

print("Count:", count)


Count: 36


##### tumbling windows and watermarking

###### window size 15 mins, watermarking 2 mins, max temp, number of drops

In [ ]:
df_B03494 = df_stream.where("hvfhs_license_num == 'B03494'")

# write to memory and select * from "topic" to inspect content or write to console
params = {
"df": df_B03494,
"spark": spark,
"topic": "nycstream_B03494",       #the in-memory df would be of this name
"write_to": "console",
"trigger_mode": "processingTime", #once, continuous
"output_path": session_vars.delta_path,
"output_mode": "update",
"merge_keys": session_vars.merge_keys,
"interval_seconds": 5,
}

query = write_spark_stream(**params)
# while running:
# print("offset just written: ",query.lastProgress)#["sources"][0]["endOffset"])

# Wait until the query finishes
try:
    query.awaitTermination()
    # spark.sql("select * from nycstream").show(10)
    # if params_write_spark_stream["trigger_mode"] == "continuous": 
    #     spark.sql("select * from nycstream").show(10)
finally:
    if params["trigger_mode"] != "continuous":
        query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------+--------------------+---------------+----------------+------------+------------+----------+---------+
|hvfhs_license_num|dispatching_base_num|pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|
+-----------------+--------------------+---------------+----------------+------------+------------+----------+---------+
+-----------------+--------------------+---------------+----------------+------------+------------+----------+---------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt


KeyboardInterrupt: 

26/01/02 12:39:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [ ]:
# on every 5 mins, with watermarking of 2 mins, record lowest temperature, and count of drops at locatioid = 258
# join on locationid with watermarking, count(*) from nyctaxi, min(temp) from temp stream





In [ ]:
# ASYNC EXAMPLE

# import asyncio

# async def worker(name):
#     print(name, "started")
#     for i in range(5):
#         print(name, "step", i)
#         await asyncio.sleep(0.5)  # yield time
#     print(name, "finished")

# async def main():
#     asyncio.create_task(worker("A"))
#     asyncio.create_task(worker("B"))
#     asyncio.create_task(worker("C"))

#     print("Main is doing something...")
#     await asyncio.sleep(3)  # yields → all tasks run
#     print("Main done")

# # asyncio.run(main())
# await main()

